In [1]:
import time

import numpy as np
import pandas as pd
import cv2
import pickle

from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model

In [2]:
labels = pd.read_csv('../data/label_names_eng.csv')
print(labels.head())

   ClassId              SignName
0        0  Speed limit (20km/h)
1        1  Speed limit (30km/h)
2        2  Speed limit (50km/h)
3        3  Speed limit (60km/h)
4        4  Speed limit (70km/h)


In [3]:
model = load_model('../models/model-new.h5')
with open('../data/mean_image_rgb.pickle', 'rb') as f:
    mean = pickle.load(f, encoding='latin1')
print(mean['mean_image_rgb'].shape)

(3, 32, 32)


In [4]:
path_to_weights: str = '../data/znaki_rtx_final.weights'
path_to_cfg: str = '../data/yolov3_ts_test.cfg'
network = cv2.dnn.readNetFromDarknet(path_to_cfg, path_to_weights)
network.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
network.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)
# network.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
# network.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL_FP16)

In [5]:
layers_all = network.getLayerNames()
print(layers_all)
layers_names_output = [layers_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]
print(layers_names_output)

['conv_0', 'bn_0', 'leaky_1', 'conv_1', 'bn_1', 'leaky_2', 'conv_2', 'bn_2', 'leaky_3', 'conv_3', 'bn_3', 'leaky_4', 'shortcut_4', 'conv_5', 'bn_5', 'leaky_6', 'conv_6', 'bn_6', 'leaky_7', 'conv_7', 'bn_7', 'leaky_8', 'shortcut_8', 'conv_9', 'bn_9', 'leaky_10', 'conv_10', 'bn_10', 'leaky_11', 'shortcut_11', 'conv_12', 'bn_12', 'leaky_13', 'conv_13', 'bn_13', 'leaky_14', 'conv_14', 'bn_14', 'leaky_15', 'shortcut_15', 'conv_16', 'bn_16', 'leaky_17', 'conv_17', 'bn_17', 'leaky_18', 'shortcut_18', 'conv_19', 'bn_19', 'leaky_20', 'conv_20', 'bn_20', 'leaky_21', 'shortcut_21', 'conv_22', 'bn_22', 'leaky_23', 'conv_23', 'bn_23', 'leaky_24', 'shortcut_24', 'conv_25', 'bn_25', 'leaky_26', 'conv_26', 'bn_26', 'leaky_27', 'shortcut_27', 'conv_28', 'bn_28', 'leaky_29', 'conv_29', 'bn_29', 'leaky_30', 'shortcut_30', 'conv_31', 'bn_31', 'leaky_32', 'conv_32', 'bn_32', 'leaky_33', 'shortcut_33', 'conv_34', 'bn_34', 'leaky_35', 'conv_35', 'bn_35', 'leaky_36', 'shortcut_36', 'conv_37', 'bn_37', 'leaky_

In [6]:
probability_minimum = 0.2
threshold = 0.2
colors = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

print(colors.shape)
print(colors[0])

(43, 3)
[ 35 120 207]


In [ ]:
image_BGR = cv2.imread('../data/yolo_formated/autosave21_01_2013_12_35_10_2.jpg')
print(f'Input shape: {image_BGR.shape}')
h, w = image_BGR.shape[:2]

blob = cv2.dnn.blobFromImage(image_BGR, 1 / 255.0, (416, 416), swapRB=True, crop=False)
network.setInput(blob)
start = time.time()
output_from_network = network.forward(layers_names_output)
end = time.time()

print(f'Total time: {end - start:.5f}')

bounding_boxes = []
confidences = []
class_numbers = []
for result in output_from_network:
    for detected_objects in result:
        scores = detected_objects[5:]
        class_current = np.argmax(scores)
        confidence_current = scores[class_current]

        if confidence_current > probability_minimum:
            box_current = detected_objects[0:4] * np.array([w, h, w, h])

            x_center, y_center, box_width, box_height = box_current
            x_min = int(x_center - (box_width / 2))
            y_min = int(y_center - (box_height / 2))

            bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
            confidences.append(float(confidence_current))
            class_numbers.append(class_current)

results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)

if len(results) > 0:
    for i in results.flatten():
        x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
        box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]


        c_ts = image_BGR[y_min:y_min+int(box_height), x_min:x_min+int(box_width), :]

        if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
            pass
        else:
            blob_ts = cv2.dnn.blobFromImage(c_ts, 1 / 255.0, size=(32, 32), swapRB=True, crop=False)
            blob_ts[0] = blob_ts[0, :, :, :] - mean['mean_image_rgb']
            blob_ts = blob_ts.transpose(0, 2, 3, 1)

            scores = model.predict(blob_ts)

            prediction = np.argmax(scores)
            print(labels['SignName'][prediction])

            colour_box_current = colors[class_numbers[i]].tolist()

            cv2.rectangle(image_BGR, (x_min, y_min), (x_min + box_width, y_min + box_height),
                          colour_box_current, 2)
            text_box_current = '{}: {:.4f}'.format(labels['SignName'][prediction], confidences[i])
            cv2.putText(image_BGR, text_box_current, (x_min, y_min - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour_box_current, 2)

cv2.imwrite('../result.png', image_BGR)


In [ ]:
%matplotlib inline

plt.rcParams['figure.figsize'] = (35.0, 35.0) # Setting default size of plots
image_BGR = cv2.imread('../result.png')
print('Image shape:', image_BGR.shape)

h, w = image_BGR.shape[:2]
print(f'Высота={h} ширина={w}')

plt.imshow(cv2.cvtColor(image_BGR, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()
plt.close()

In [7]:
video = cv2.VideoCapture(0)
writer = None
h, w = None, None


In [8]:
%matplotlib inline

plt.rcParams['figure.figsize'] = (3, 3)
f = 0
t = 0

while True:
    ret, frame = video.read()
    if not ret:
        break

    if w is None or h is None:
        h, w = frame.shape[:2]

    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)

    network.setInput(blob)
    start = time.time()
    output_from_network = network.forward(layers_names_output)
    end = time.time()

    f += 1
    t += end - start
    print('Frame number {0} took {1:.5f} seconds'.format(f, end - start))

    bounding_boxes = []
    confidences = []
    class_numbers = []

    for result in output_from_network:
        for detected_objects in result:
            scores = detected_objects[5:]
            class_current = np.argmax(scores)
            confidence_current = scores[class_current]
            if confidence_current > probability_minimum:
                box_current = detected_objects[0:4] * np.array([w, h, w, h])

                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)

    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)

    if len(results) > 0:
        for i in results.flatten():
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

            c_ts = frame[y_min:y_min+int(box_height), x_min:x_min+int(box_width), :]

            if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
                pass
            else:
                blob_ts = cv2.dnn.blobFromImage(c_ts, 1 / 255.0, size=(32, 32), swapRB=True, crop=False)
                blob_ts[0] = blob_ts[0, :, :, :] - mean['mean_image_rgb']
                blob_ts = blob_ts.transpose(0, 2, 3, 1)

                scores = model.predict(blob_ts)
                prediction = np.argmax(scores)

                colour_box_current = colors[class_numbers[i]].tolist()

                cv2.rectangle(frame, (x_min, y_min), (x_min + box_width, y_min + box_height),
                              colour_box_current, 2)

                text_box_current = '{}: {:.4f}'.format(labels['SignName'][prediction], confidences[i])
                cv2.putText(frame, text_box_current, (x_min, y_min - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour_box_current, 2)

    cv2.namedWindow('Camera view', cv2.WINDOW_NORMAL)
    cv2.imshow('Camera view', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


Frame number 1 took 0.81400 seconds
Frame number 2 took 0.43000 seconds
Frame number 3 took 0.40799 seconds
Frame number 4 took 0.38199 seconds
Frame number 5 took 0.51000 seconds
Frame number 6 took 0.53801 seconds
Frame number 7 took 0.42500 seconds
Frame number 8 took 0.48300 seconds
Frame number 9 took 0.44500 seconds
Frame number 10 took 0.39900 seconds
Frame number 11 took 0.38900 seconds
Frame number 12 took 0.38200 seconds
Frame number 13 took 0.38200 seconds
Frame number 14 took 0.38199 seconds
Frame number 15 took 0.41300 seconds
Frame number 16 took 0.38100 seconds
Frame number 17 took 0.38600 seconds


In [14]:
print('Всего обработано кадров', f)
print(f'Заняло {t:.5f} секунд')
print('FPS:', round((f / t), 1))

Всего обработано кадров 257
Заняло 106.47058 секунд
FPS: 2.4
